<a href="https://colab.research.google.com/github/imabari/ImabariScraping/blob/master/Invisible_Intersection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 都道府県コード

https://www.npa.go.jp/publications/statistics/koutsuu/opendata/2021/codebook_2021.pdf

In [ ]:
pref = 45 #@param {type:"integer"}


# 日本測地系2011（JGD2011）

https://lemulus.me/column/epsg-list-gis#i

In [ ]:
epsg = 6677 #@param {type:"integer"}


In [ ]:
!pip install geopandas

# プログラム

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
from shapely.geometry import Point

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_columns", None)

In [ ]:
df0 = pd.read_csv("https://www.npa.go.jp/publications/statistics/koutsuu/opendata/2021/honhyo_2021.csv", encoding="cp932")

In [ ]:
# 緯度経度を度分秒から10進数表記に変換

def dms2deg(df_tmp, col):

    df = df_tmp.copy()
    
    df["d"], df["t"] = df[col].divmod(10000000)
    df["m"], df["s"] = df["t"].divmod(100000)

    result = df["d"] + (df["m"] / 60.0) + (df["s"] / 1000.0 / 3600.0 )

    return result

In [ ]:
df0["lat"] = dms2deg(df0, "地点　緯度（北緯）")
df0["lon"] = dms2deg(df0, "地点　経度（東経）")

In [ ]:
df0

In [ ]:
df0.columns

In [ ]:
df0["発生日時　　年"].value_counts()

In [ ]:
# 2021年のみ
df1 = df0[df0["発生日時　　年"] == 2021].copy()

In [ ]:
df1.shape

In [ ]:
# 月ごとの事故数を確認
df1["発生日時　　月"].value_counts()

In [ ]:
# 曜日別の事故数を確認
df1["曜日(発生年月日)"].value_counts()

In [ ]:
# 昼夜別の事故数を確認
df1["昼夜"].value_counts()

In [ ]:
# 道路の形ごとの事故数を確認
df1["道路線形"].value_counts()

In [ ]:
# 交差点の大きさごとの事故数を確認
df1["車道幅員"].value_counts()

In [ ]:
df2 = df1[df1["車道幅員"].isin([11, 14, 15]) & (df1["都道府県コード"] == pref)]
df2

In [ ]:
geo_df = gpd.GeoDataFrame(df2, geometry = gpd.points_from_xy(df2.lon, df2.lat), crs=epsg)
geo_df

In [ ]:
geo_df.crs

In [ ]:
df3 = geo_df.copy()

In [ ]:
# 0.0001度　約11m
df3["buffer"] = df3.buffer(0.0001)

In [ ]:
df3

In [ ]:
for i, r in tqdm(df3.iterrows()):
    
    dft = geo_df[geo_df.geometry.within(r.buffer)].copy().sort_index()

    df3.at[i, "len"] = len(dft)
    df3.at[i, "idx"] = "_".join(map(str, dft.index.values))

In [ ]:
"""
for i, r in tqdm(df3.iterrows()):
    
    point = Point(r.lon, r.lat)

    # 10mの範囲
    buffer = point.buffer(0.0001)

    tmp = geo_df[geo_df.geometry.within(buffer)].copy()

    circles = tmp.geometry.buffer(0.0001).unary_union

    dft = geo_df[geo_df.geometry.within(circles)].copy().sort_index()

    df3.at[i, "len"] = len(dft)
    df3.at[i, "idx"] = "_".join(map(str, dft.index.values))
"""

In [ ]:
df3

In [ ]:
df3["len"].value_counts()

In [ ]:
df4 = df3[df3["len"] > 5].copy()

In [ ]:
df4

In [ ]:
se = df4.groupby("idx").apply(len).rank(ascending=False).astype(int).apply(lambda x: f"intersection_{x}")
se.name = "intersection_name"

In [ ]:
df5 = pd.merge(df4, se, on="idx")

In [ ]:
df5

# 地図

In [ ]:
import folium
from folium import plugins

In [ ]:
m = folium.Map(
    location=[df5.lat.mean(), df5.lon.mean()],
    zoom_start=12,
    tiles="https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png",
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">国土地理院</a>',
)

In [ ]:
for i, r in df5.iterrows():

    folium.Marker(
        location=[r.lat, r.lon],
        icon=folium.Icon(color="red")
    ).add_to(m)

In [ ]:
# DRAW
folium.plugins.Draw(
    draw_options={"polygon": False, "rectangle": False, "circlemarker": False}
).add_to(m)

In [ ]:
m